In [1]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person, SnapshotObjectType, OperationStatusType
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.environ['FACE_SUBSCRIPTION_KEY'] = '5df507ddf5d84572b4c2c99d39879490'
os.environ['FACE_ENDPOINT'] = 'https://faceapiappzhaoqc.cognitiveservices.azure.com/'
remote_path = '/store/Data/AttendanceData/temp.png'
local_path = '/Users/zhaoqc/Program/Docker/age_gender_recognition/aragaki_yokogao.jpeg'

KEY = os.environ['FACE_SUBSCRIPTION_KEY']
ENDPOINT = os.environ['FACE_ENDPOINT']
print(KEY)
print(ENDPOINT)

5df507ddf5d84572b4c2c99d39879490
https://faceapiappzhaoqc.cognitiveservices.azure.com/


In [3]:
face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

In [4]:
single_face_image_url = 'https://www.biography.com/.image/t_share/MTQ1MzAyNzYzOTgxNTE0NTEz/john-f-kennedy---mini-biography.jpg'
single_image_name = os.path.basename(single_face_image_url)
detected_faces = face_client.face.detect_with_url(url=single_face_image_url)
if not detected_faces:
    raise Exception('No face detected from image {}'.format(single_image_name))

# Display the detected face ID in the first single-face image.
# Face IDs are used for comparison to faces (their IDs) detected in other images.
print('Detected face ID from', single_image_name, ':')
for face in detected_faces: 
    print(face.face_id)
print()

# Save this ID for use in Find Similar
first_image_face_ID = detected_faces[0].face_id

def getRectangle(faceDictionary):
    rect = faceDictionary.face_rectangle
    left = rect.left
    top = rect.top
    bottom = left + rect.height
    right = top + rect.width
    return ((left, top), (bottom, right))



Detected face ID from john-f-kennedy---mini-biography.jpg :
33ed42a5-40ca-4922-8502-a832ec130388



In [50]:
# Detect the faces in an image that contains multiple faces
# Each detected face gets assigned a new ID
multi_face_image_url = "http://www.historyplace.com/kennedy/president-family-portrait-closeup.jpg"
multi_image_name = os.path.basename(multi_face_image_url)
detected_faces2 = face_client.face.detect_with_url(url=multi_face_image_url)
second_image_face_IDs = list(map(lambda x: x.face_id, detected_faces2))
similar_faces = face_client.face.find_similar(face_id=first_image_face_ID, face_ids=second_image_face_IDs)
print('Similar faces found in', multi_image_name + ':')
for face in similar_faces:
    first_image_face_ID = face.face_id
    # The similar face IDs of the single face image and the group image do not need to match, they are only used for identification purposes in each image.
    # The similar faces are matched using the Cognitive Services algorithm in find_similar().
    face_info = next(x for x in detected_faces2 if x.face_id == first_image_face_ID)
    if face_info:
        print('  Face ID: ', first_image_face_ID)
        print('  Face rectangle:')
        print('    Left: ', str(face_info.face_rectangle.left))
        print('    Top: ', str(face_info.face_rectangle.top))
        print('    Width: ', str(face_info.face_rectangle.width))
        print('    Height: ', str(face_info.face_rectangle.height))

Similar faces found in president-family-portrait-closeup.jpg:
  Face ID:  15019d2e-d7d0-4985-bd4b-845df39a9b73
  Face rectangle:
    Left:  413
    Top:  57
    Width:  55
    Height:  55


In [55]:
response = requests.get(multi_face_image_url)
img = Image.open(BytesIO(response.content))
draw = ImageDraw.Draw(img)

for face in similar_faces:
    face_info = next(x for x in detected_faces2 if x.face_id == first_image_face_ID)
    left_top = (face_info.face_rectangle.left,face_info.face_rectangle.top)
    bottom_right = (face_info.face_rectangle.left+face_info.face_rectangle.width,face_info.face_rectangle.top+face_info.face_rectangle.height)
    draw.rectangle((left_top,bottom_right), outline='red')
    
img.show()

In [33]:
face_client.person_group.delete(person_group_id=PERSON_GROUP_ID)

In [34]:
# Used in the Person Group Operations,  Snapshot Operations, and Delete Person Group examples.
# You can call list_person_groups to print a list of preexisting PersonGroups.
# SOURCE_PERSON_GROUP_ID should be all lowercase and alphanumeric. For example, 'mygroupname' (dashes are OK).
PERSON_GROUP_ID = 'my-unique-person-group'

# Used for the Snapshot and Delete Person Group examples.
TARGET_PERSON_GROUP_ID = str(uuid.uuid4()) # assign a random ID (or name it anything)

# Create empty Person Group. Person Group ID must be lower case, alphanumeric, and/or with '-', '_'.
print('Person group:', PERSON_GROUP_ID)
face_client.person_group.create(person_group_id=PERSON_GROUP_ID, name=PERSON_GROUP_ID)

# Define woman friend
woman = face_client.person_group_person.create(PERSON_GROUP_ID, "Woman")
# Define man friend
huffman = face_client.person_group_person.create(PERSON_GROUP_ID, "Huffman")
# Define child friend
tatsumi = face_client.person_group_person.create(PERSON_GROUP_ID, "Tatsumi")

Person group: my-unique-person-group


In [101]:
'''
Detect faces and register to correct person
'''
# Find all jpeg images of friends in working directory
woman_images = [file for file in glob.glob('*.jpg') if file.startswith("women")]
huffman_images = [file for file in glob.glob('*.jpg') if file.startswith("huffman")]
tatsumi_images = [file for file in glob.glob('*.jpg') if file.startswith("tatsumi")]

# Add to a woman person
for image in woman_images:
    w = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, woman.person_id, w)

# Add to a man person
for image in huffman_images:
    m = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, huffman.person_id, m)

# Add to a child person
for image in tatsumi_images:
    ch = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, tatsumi.person_id, ch)

In [102]:
'''
Train PersonGroup
'''
print()
print('Training the person group...')
# Train the person group
face_client.person_group.train(PERSON_GROUP_ID)

while (True):
    training_status = face_client.person_group.get_training_status(PERSON_GROUP_ID)
    print("Training status: {}.".format(training_status.status))
    print()
    if (training_status.status is TrainingStatusType.succeeded):
        break
    elif (training_status.status is TrainingStatusType.failed):
        sys.exit('Training the person group has failed.')
    time.sleep(5)


Training the person group...
Training status: running.

Training status: succeeded.



In [104]:
'''
Identify a face against a defined PersonGroup
'''
# Group image for testing against
group_photo = 'test.jpg'
IMAGES_FOLDER = './'
# Get test image
test_image_array = glob.glob(os.path.join(IMAGES_FOLDER, group_photo))
image = open(test_image_array[0], 'r+b')

# Detect faces
face_ids = []
faces = face_client.face.detect_with_stream(image)
for face in faces:
    face_ids.append(face.face_id)
    
print(face_ids)

['42c481d3-116d-47d3-b1af-c3a8af5df6b2']


In [108]:
for person in face_client.person_group_person.list(PERSON_GROUP_ID):
    print(person.person_id, person.name)

106f03bf-ea92-4599-ae8c-8ba23d126e2c Tatsumi
6d7b2857-77cd-45c8-9b02-4de94700726a Huffman
aef51426-060a-4192-8ab4-e31a42133a35 Woman


In [105]:
# Identify faces
results = face_client.face.identify(face_ids, PERSON_GROUP_ID)

print('Identifying faces in {}'.format(os.path.basename(image.name)))
if not results:
    print('No person identified in the person group for faces from {}.'.format(os.path.basename(image.name)))
for person in results:
    print('Person for face ID {} is identified as {} with a confidence of {}.'.format(person.face_id, person.candidates[0].person_id,person.candidates[0].confidence)) # Get topmost confidence score

Identifying faces in test.jpg
Person for face ID 42c481d3-116d-47d3-b1af-c3a8af5df6b2 is identified as aef51426-060a-4192-8ab4-e31a42133a35 with a confidence of 0.54961.


In [42]:
import paramiko

ssh = paramiko.SSHClient() 
ssh.load_host_keys(os.path.expanduser(os.path.join("~", ".ssh", "known_hosts")))
ssh.connect('10.1.103.21', username='zhaoqc', password='4425091zqc')
sftp = ssh.open_sftp()
sftp.chdir('/store/Data/AttendanceData/')
# (stdin, stdout, stderr) = ssh.exec_command('pwd')
# print(stdout.read().decode())
sftp.put(local_path,'./temp2.png')
sftp.close()
ssh.close()


FileNotFoundError: [Errno 2] No such file or directory: 'https://www.biography.com/.image/t_share/MTQ1MzAyNzYzOTgxNTE0NTEz/john-f-kennedy---mini-biography.jpg'